# Homography estimation for Kornia DoG implementation in Kornia API

* An attempt to replicate 'Kornia homography estimation OpenCV API.ipynb' notebook for Kornia DoG using Kornia/torch API only.

### Performed on 4 datasets

* boat and bark datasets from https://www.robots.ox.ac.uk/~vgg/data/affine/
* pure rotations of the first image from "bark" by multiples of 90 degrees
* pure scaling of the first image from "bark" by multiples of 0.1 adjusted so that the aspect ratio is preserved

* probably due to the implementation of kornia.geometry RANSAC it's much less stable - it needs a very relaxed
  set of parameters (max_iter, confidence) to finish in some feasible time. 
* see "./Kornia homography estimation OpenCV API.ipynb"
* see https://github.com/kornia/kornia-benchmark/blob/master/local-features/DoG%20precision.ipynb


In [2]:
from kornia.feature import match_mnn
from kornia.feature.integrated import SIFTFeature
from kornia.geometry import RANSAC

from dataset_utils import *
from kornia_utils import *


def homography_estimation_experiment(Hs_gt_t, imgs_t, device, e_name, instance_names):

    print(f"\nrunning experiment: {e_name}")
    print("\t\terror \t\t\t\t #tentatives \t\t #inliers")

    ransac = RANSAC(
        model_type='homography', inl_th=0.5, batch_size=2048, max_iter=100, confidence=0.95, max_lo_iters=5
    )

    sf = SIFTFeature(device=device)
    lafs0, responses0, descs_t0 = sf(imgs_t[0], mask=None)

    sum_reproj_err = 0.0
    sum_tent_count = 0
    sum_in_count = 0

    for other_i in range(1, len(imgs_t)):
        lafs1, responses1, descs_t1 = sf(imgs_t[other_i], mask=None)
        _, matches = match_mnn(descs_t0[0], descs_t1[0], dm=None)
        src_pts_t = lafs0[0, matches[:, 0], :, 2]
        dst_pts_t = lafs1[0, matches[:, 1], :, 2]

        if len(src_pts_t) < 4:
            print(f"WARNING: less than 4 tentatives: {len(src_pts_t)}")
            continue

        H_est_t, inliers = ransac(src_pts_t, dst_pts_t)

        H_gt_t = Hs_gt_t[other_i - 1]
        reproj_err = get_visible_part_mean_absolute_reprojection_error_torch(
            imgs_t[0], imgs_t[other_i], H_gt_t, H_est_t, device
        )

        tent_count = len(src_pts_t)
        in_count = int(inliers.sum())

        print(f"{instance_names[other_i - 1]}\t\t{reproj_err:.6f} \t\t\t {tent_count} \t\t\t {in_count}")
        sum_reproj_err += reproj_err
        sum_tent_count += tent_count
        sum_in_count += in_count

    print(f"Sum:\t\t{sum_reproj_err:.6f} \t\t\t {sum_tent_count} \t\t\t {sum_in_count}")


device = torch.device("cpu")

Hs_boat, imgs_boat = Hs_imgs_for_boat_torch(device)
Hs_bark, imgs_bark = Hs_imgs_for_bark_torch(device)
Hs_rot, imgs_rot = Hs_imgs_for_rotation_torch(device)
Hs_scaling, imgs_scaling, scales = Hs_imgs_for_scaling_torch(device)

homography_estimation_experiment(Hs_rot, imgs_rot, device, "synthetic pi rotation", ["90deg", "180deg", "270deg"])
homography_estimation_experiment(Hs_bark, imgs_bark, device, "bark", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(Hs_boat, imgs_boat, device, "boat", [f"img{i}" for i in range(2, 7)])
homography_estimation_experiment(Hs_scaling, imgs_scaling, device, "synthetic rescaling lanczos", [f"{s}" for s in scales])


scale: 0.2 => 0.2
scale: 0.3 => 0.2980392156862745
scale: 0.4 => 0.4
scale: 0.5 => 0.5019607843137255
scale: 0.6 => 0.6
scale: 0.7 => 0.6980392156862745
scale: 0.8 => 0.8
scale: 0.9 => 0.9019607843137255

running experiment: synthetic pi rotation
		error 				 #tentatives 		 #inliers
90deg		0.492788 			 6676 			 5864
180deg		0.706998 			 6469 			 5473
270deg		0.501235 			 6594 			 5758
Sum:		1.701021 			 19739 			 17095

running experiment: bark
		error 				 #tentatives 		 #inliers
img2		0.724287 			 1507 			 287
img3		1.766101 			 1072 			 229
img4		1.180466 			 1133 			 472
img5		0.759804 			 930 			 305
img6		1.121154 			 717 			 143
Sum:		5.551813 			 5359 			 1436

running experiment: boat
		error 				 #tentatives 		 #inliers
img2		0.267608 			 3033 			 1113
img3		0.171984 			 2315 			 956
img4		1.187037 			 1401 			 260
img5		0.360316 			 1220 			 138
img6		5.764807 			 790 			 52
Sum:		7.751753 			 8759 			 2519

running experiment: synthetic rescaling lanczos
		error 				 #tenta